In [1]:
import pandas as pd
import numpy as np

In [2]:
names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('ml-100k/u.data',sep='\t',names=names)

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols, encoding='latin-1')


In [3]:
user_item_df = np.zeros((len(df['user_id'].unique()),len(items)))

for x in df.itertuples():
    user_item_df[x[1]-1,x[2]-1] = x[3]
    
user_item_df = pd.DataFrame(user_item_df)

In [5]:
from sklearn.model_selection import train_test_split
train_df,test_df = train_test_split(df, test_size=0.2)

# Training Dataset
train_ds = np.zeros((len(df['user_id'].unique()),len(items)))
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
train_ds = pd.DataFrame(train_ds)

# Testing Dataset
test_ds = np.zeros((len(df['user_id'].unique()),len(items)))
for row in test_df.itertuples():
    test_ds[row[1]-1, row[2]-1] = row[3]
test_ds = pd.DataFrame(test_ds)

(     0     1     2     3     4     5     6     7     8     9     ...  1672  \
 0     5.0   3.0   0.0   0.0   3.0   5.0   4.0   1.0   5.0   3.0  ...   0.0   
 1     4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   2.0  ...   0.0   
 2     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 3     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 4     0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 ..    ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
 938   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   5.0   0.0  ...   0.0   
 939   0.0   0.0   0.0   0.0   0.0   0.0   4.0   5.0   3.0   0.0  ...   0.0   
 940   5.0   0.0   0.0   0.0   0.0   0.0   4.0   0.0   0.0   0.0  ...   0.0   
 941   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   0.0   
 942   0.0   5.0   0.0   0.0   0.0   0.0   0.0   0.0   3.0   0.0  ...   0.0   
 
      1673  1674  1675  1676  1677  1678  1679  16

In [7]:
GAMMA = 30
EPSILON = 1e-9

pearson_mat = np.zeros((len(df['user_id'].unique()),len(df['user_id'].unique())))

for i,user_vector1 in enumerate(train_ds.values):
    for j,user_vector2 in enumerate(train_ds.values):
        
        mask_i = user_vector1>0
        mask_j = user_vector2>0
        
        corratedItems = np.intersect1d(np.where(mask_i),np.where(mask_j))
        
        if len(corratedItems)==0:
            continue
        
        user_masked_vector1 = user_vector1[mask_i]
        user_masked_vector2 = user_vector2[mask_j]
        
        mean_vector1 = np.sum(user_masked_vector1)/(np.sum(np.clip(user_vector1,0,1))+EPSILON)
        mean_vector2 = np.sum(user_masked_vector2)/(np.sum(np.clip(user_vector2,0,1))+EPSILON)
        
        subtracted_i = user_vector1[corratedItems]-mean_vector1
        subtracted_j = user_vector2[corratedItems]-mean_vector2
        
        numerator = subtracted_i * subtracted_j
        numerator = np.sum(numerator)
        
        squared_i = np.square(subtracted_i)
        squared_j = np.square(subtracted_j)
        
        sqrt_i = np.sqrt(np.sum(squared_i))
        sqrt_j = np.sqrt(np.sum(squared_j))
        
        denominator = sqrt_i*sqrt_j
        
        sim = numerator/(denominator+EPSILON)
        
        pearson_mat[i][j] = (min(len(corratedItems),GAMMA)/GAMMA)*sim

In [9]:
pearson_mat

array([[ 1.        ,  0.14532078, -0.01258684, ...,  0.04132055,
        -0.0238206 ,  0.14876828],
       [ 0.14532078,  1.        ,  0.08897483, ..., -0.01289529,
         0.01305923,  0.13834429],
       [-0.01258684,  0.08897483,  1.        , ...,  0.10582073,
        -0.03550603,  0.03333333],
       ...,
       [ 0.04132055, -0.01289529,  0.10582073, ...,  0.6       ,
        -0.02763688,  0.0165001 ],
       [-0.0238206 ,  0.01305923, -0.03550603, ..., -0.02763688,
         1.        ,  0.15513162],
       [ 0.14876828,  0.13834429,  0.03333333, ...,  0.0165001 ,
         0.15513162,  1.        ]])

In [10]:
k=100
np_predictions = np.zeros((len(df['user_id'].unique()),len(items)))

for (i,j),rating in np.ndenumerate(test_ds.values):
    if rating>0:
        
        sim_users_id = np.argsort(pearson_mat[i])[-k:]
        sim_val = pearson_mat[i][sim_users_id]
        sim_users = train_ds.values[sim_users_id]
        
        curr_user_mean = np.sum(train_df.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
        sim_users_mean = np.sum(sim_users,axis=1)/ (np.sum(np.clip(sim_users, 0, 1),axis=1) + EPSILON)
        
        masked_j = sim_users[:,j]>0
        
        subtracted_mean = sim_users[masked_j,j]-sim_users_mean[masked_j]
        
        numerator = np.sum(sim_val[masked_j]*subtracted_mean)
        denominator = np.sum(sim_val[masked_j])
        
        np_predictions[i][j] = curr_user_mean + (numerator/(denominator+EPSILON))
        np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)
        

In [13]:
from sklearn.metrics import mean_squared_error
mean_squared_error(np_predictions,test_ds.values)

0.04282131432181302